<a href="https://colab.research.google.com/github/Tambup/wirenet-again/blob/main/DeepExplainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install shap

In [20]:
import pickle
import shap

In [21]:
import torch
import torch.nn
import torch.nn.functional as F

In [22]:
prefix = '/content/drive/MyDrive/ember2018'

In [23]:
class MalConv_ForADV(torch.nn.Module):
  # trained to minimize cross-entropy loss
  # criterion = nn.CrossEntropyLoss()
  def __init__(self, out_size=2, channels=128, window_size=512, embd_size=8):
    super(MalConv_ForADV, self).__init__()
    self.embd = torch.nn.Embedding(257, embd_size, padding_idx=0)
    self.embd.requires_grad_(True)
    self.window_size = window_size

    self.conv_1 = torch.nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)
    self.conv_2 = torch.nn.Conv1d(embd_size, channels, window_size, stride=window_size, bias=True)

    self.pooling = torch.nn.AdaptiveMaxPool1d(1)

    self.fc_1 = torch.nn.Linear(channels, channels)
    self.fc_2 = torch.nn.Linear(channels, out_size)

  def forward(self, x):
    # print('Input:', x.shape)
    x = torch.transpose(x, -1, -2)
    # print('Post transpose:', x.shape)
    cnn_value = self.conv_1(x)
    gating_weight = torch.sigmoid(self.conv_2(x))
    # print('Post sigm cnn:', cnn_value.shape)
    # print('Post sigm gating_weight:', gating_weight.shape)
    x = cnn_value * gating_weight
    # print('Post xor:', x.shape)
    x = self.pooling(x)
    # print('Post pooling:', x.shape)

    # Flatten
    x = x.view(x.size(0), -1)
    # print('Post flatten:', x.shape)
    x = F.relu(self.fc_1(x))
    # print('Post ReLu fc1:', x.shape)
    x = self.fc_2(x)
    # print('Post fc2:', x.shape)
    x = F.softmax(x, dim=1)
    return x  

# Primi kb 

In [ ]:
with open(f'{prefix}/pickles/deep_explainer.pickle', 'rb') as handle:
    explainer = pickle.load(handle)

In [46]:
with open(f'{prefix}/pickles/deep_shap_bad.pickle', 'rb') as handle:
    shap_values_bad = pickle.load(handle)
with open(f'{prefix}/pickles/deep_shap.pickle', 'rb') as handle:
    shap_values = pickle.load(handle)

In [78]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0])

In [79]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values_bad[1][0])

# Ultimi kb

In [62]:
with open(f'{prefix}/pickles/end_deep_explainer.pickle', 'rb') as handle:
    end_explainer = pickle.load(handle)

In [63]:
with open(f'{prefix}/pickles/end_deep_shap_bad.pickle', 'rb') as handle:
    end_shap_values_bad = pickle.load(handle)
with open(f'{prefix}/pickles/end_deep_shap.pickle', 'rb') as handle:
    end_shap_values = pickle.load(handle)

In [77]:
shap.initjs()
shap.force_plot(end_explainer.expected_value[1], end_shap_values[1][0])

In [89]:
shap.initjs()
shap.force_plot(end_explainer.expected_value[1], end_shap_values_bad[1][0])